In [1]:
from gensim import downloader
from gensim.similarities.annoy import AnnoyIndexer
import os
import json
import gradio as gr

In [2]:
target_words = dict()
with open('./targets.json', 'r') as file:
    target_words = json.load(file)

In [3]:
print("Downloading word2vec model", end="\r")

model = downloader.load('word2vec-google-news-300')

print("Downloaded word2vec model ")

fname = './tmp/mymodel.index'

if os.path.exists(fname):
    print("Loading random forest for approximate nearest neighbors")
    annoy_index = AnnoyIndexer()
    annoy_index.load(fname)
    annoy_index.model = model
else:
    print("Creating random forest for approximate nearest neighbors")   
    # 100 trees are being used in this example
    annoy_index = AnnoyIndexer(model, 10)
    # Persist index to disk
    annoy_index.save(fname)

print("All set!")

Downloaded word2vec model 
Loading random forest for approximate nearest neighbors
All set!


In [4]:
input_values = []

def on_text_submit(guess: str, is_junior: bool):
    
    # get the target word
    target_level = "junior" if is_junior else "normal"
    target = target_words[target_level]
    
    output_str = f"({len(input_values) + 1})"
    if guess == target:
        input_values.insert(0, f"{output_str} SUCCESS {target}!")
    else:
        try:
            approx_direction = model[target] - model[guess]
            approximate_neighbors = model.most_similar([approx_direction], topn=3, indexer=annoy_index)
            approximate_neighbors_str = " ".join([x[0].lower() for x in approximate_neighbors])
            input_values.insert(0, f"{output_str} {guess}: {approximate_neighbors_str}")
        except Exception:
            input_values.insert(0, f"{output_str} {guess} not found")
    
    return "\n".join(input_values)

In [6]:
demo = gr.Interface(
    fn=on_text_submit,
    inputs=["text", "checkbox"],
    outputs="text",
    )
    
demo.launch()

Running on local URL:  http://127.0.0.1:7861

To create a public link, set `share=True` in `launch()`.
